In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.4.0
Populating the interactive namespace from numpy and matplotlib


# Pandas Datareader Test

In [2]:
cd /Users/jinsungpark/Desktop/Data_river/data04

[Errno 2] No such file or directory: '/Users/jinsungpark/Desktop/Data_river/data04'
/Users/jinsungpark/Desktop/jupyter


In [3]:
pwd #현재 경로 확인

'/Users/jinsungpark/Desktop/jupyter'

In [4]:
ls #현재경로에 있는 항목 확인

Correlation_analysis.ipynb
Data_Preprocessing.ipynb
Data_river/
Lab10_Stock_Price_Prediction_with_LSTM.ipynb
Pandas_tutorial.ipynb
Project_Git/
Project_River_Backup.ipynb
Project_River_matplotlib.ipynb
Project_River_matplotlib_01.ipynb
Untitled.ipynb
results/


In [5]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/data04

/Users/jinsungpark/Desktop/jupyter/Data_river/data04


In [6]:
UpStream_data = pd.read_excel('DS_Data_edit_log.xlsx', index = False)
DownStream_data = pd.read_excel('NG_Data_edit_log.xlsx', index = False)

In [7]:
print(UpStream_data)
# print(DownStream_data.columns)

           Date     DS_DO    DS_BOD    DS_COD     DS_SS     DS_TN     DS_TP  \
0    2013-01-07  2.782630  0.523025  1.439094  1.488231  1.187818 -3.488904   
1    2013-01-14  2.785011  0.530628  1.435085  1.386294  1.187538 -3.506558   
2    2013-01-21  2.740840  0.641854  1.504077  2.186051  1.230225 -3.575551   
3    2013-01-28  2.681022  0.336472  1.458615  1.308333  1.298555 -3.270169   
4    2013-02-04  2.694627 -0.356675  1.280934  1.629241  1.242424 -3.270169   
..          ...       ...       ...       ...       ...       ...       ...   
344  2019-09-02  2.197225  0.470004  1.648659  1.609438  0.251537 -3.442019   
345  2019-09-09  2.186051  0.530628  1.648659  1.722767  0.121332 -3.912023   
346  2019-09-16  2.041220  0.530628  1.722767  2.219203  0.131905 -3.442019   
347  2019-09-24  2.219203  0.587787  1.916923  3.054001  0.723191 -2.128632   
348  2019-09-30  2.240710  0.641854  2.054124  2.104134  0.909870 -2.385967   

     DS_Chl_a  DS_Cells   GJ_Deep  ...  GumHo_DO  G

In [8]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [9]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DS_DO              349 non-null    float64
 1   DS_BOD             349 non-null    float64
 2   DS_COD             349 non-null    float64
 3   DS_SS              349 non-null    float64
 4   DS_TN              349 non-null    float64
 5   DS_TP              349 non-null    float64
 6   DS_Chl_a           349 non-null    float64
 7   DS_Cells           349 non-null    float64
 8   GJ_Deep            349 non-null    float64
 9   GJ_Level           349 non-null    float64
 10  GJ_Outflow         349 non-null    float64
 11  DaeGu_Rain         349 non-null    float64
 12  DaeGu_Solar        349 non-null    float64
 13  SeoBu_COD          349 non-null    float64
 14  SeoBu_SS           349 non-null    float64
 15  SeoBu_TN           349 non-null    float64
 16  SeoBu_TP       

In [10]:
#넣고싶은 상류 항목 컬럼 선택
UpData = UpData.iloc[:,[0,2,3,5,10,11,28,29,33,36,38,40,41,42]]
print(UpData.columns)

Index(['DS_DO', 'DS_COD', 'DS_SS', 'DS_TP', 'GJ_Outflow', 'DaeGu_Rain',
       'SungSeo_Flow_day', 'SungSeo_COD_load', 'GumHo_DO', 'GumHo_SS',
       'GumHo_TP', 'GumHo_Flow', 'DS_Temp', 'GumHo_Temp'],
      dtype='object')


In [11]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NG_DO     349 non-null    float64
 1   NG_BOD    349 non-null    float64
 2   NG_COD    349 non-null    float64
 3   NG_SS     349 non-null    float64
 4   NG_TN     349 non-null    float64
 5   NG_TP     349 non-null    float64
 6   NG_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8+ KB


In [12]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 5
print(DownData.columns[Colum])

NG_TP


### 1.2 Data Preprocessing(normalization)

In [13]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [14]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DS_DO               0
DS_COD              0
DS_SS               0
DS_TP               0
GJ_Outflow          0
DaeGu_Rain          0
SungSeo_Flow_day    0
SungSeo_COD_load    0
GumHo_DO            0
GumHo_SS            0
GumHo_TP            0
GumHo_Flow          0
DS_Temp             0
GumHo_Temp          0
dtype: int64


# Data Preparation

In [15]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, y

# Model Define

In [16]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [17]:
# def metric(y_pred, y_true):
#     perc_y_pred = y_pred.cpu().detach().numpy()
#     perc_y_true = y_true.cpu().detach().numpy()
#     mae = mean_absolute_error(perc_y_true, perc_y_pred, multioutput='raw_values')
#     return mae*100

In [18]:
# def metric(y_pred, y_true):
#     perc_y_pred = y_pred.cpu().detach().numpy()
#     perc_y_true = y_true.cpu().detach().numpy()
#     mse = mean_squared_error(perc_y_true, perc_y_pred, multioutput='raw_values')
#     return np.sqrt(mse)

In [19]:
def metric(y_pred, y_true):
    perc_y_pred = y_pred.cpu().detach().numpy()
    perc_y_true = y_true.cpu().detach().numpy()
    
    perc_y_pred = np.exp(DeNormalize(perc_y_pred, DownData, Colum, DownScaler))
    perc_y_true = np.exp(DeNormalize(perc_y_true, DownData, Colum, DownScaler))
    
    mse = mean_squared_error(perc_y_true, perc_y_pred, multioutput='raw_values')
    return np.sqrt(mse)

# Train, Validate, Test 

In [20]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'], 
                             batch_size=args.batch_size, 
                             shuffle=True, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    train_acc = 0.0
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:, :, Colum].float().to(args.device)

        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += metric(y_pred, y_true)[0]

    train_loss = train_loss / len(trainloader)
    train_acc = train_acc / len(trainloader)
    return model, train_loss, train_acc

In [21]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    val_acc = 0.0
    val_loss = 0.0
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            val_acc += metric(y_pred, y_true)[0]

    val_loss = val_loss / len(valloader)
    val_acc = val_acc / len(valloader)
    return val_loss, val_acc

In [22]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    test_acc = 0.0
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            test_acc += metric(y_pred, y_true)[0]

    test_acc = test_acc / len(testloader)
    return test_acc

In [23]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)
    loss_fn = torch.nn.MSELoss()

    loss_fn = nn.MSELoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ===================================== #
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        model, train_loss, train_acc = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(model, partition, args)    

    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    print(type(result))
    return vars(args), result

# Manage Experiment

In [24]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [ ]:
def plot_acc(var1, var2, df):

    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    sns.barplot(x=var1, y='test_acc', hue=var2, data=df, ax=ax[2])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    ax[2].set_title('Test Accuracy')

    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) # 만약 Title이 그래프랑 겹친다면 top 값을 조정해주면 됩니다! 함수 인자로 받으면 그래프마다 조절할 수 있겠죠?


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

In [ ]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp2_lr"
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 2
args.x_frames = 5
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 25
args.n_layers = 1

# ====== Regularization ======= #
args.l2 = 0.00001
args.dropout = 0.1
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 250


# ====== Experiment Variable ====== #
name_var1 = 'hid_dim'
name_var2 = 'n_layers'
list_var1 = [25, 50, 75]
list_var2 = [1,2,3]


trainset = RiverDataset(UpData, DownData, args.x_frames, args.y_frames, '2013-01-01', '2017-12-31')
valset = RiverDataset(UpData, DownData, args.x_frames, args.y_frames, '2018-01-01', '2018-12-31')
testset = RiverDataset(UpData, DownData, args.x_frames, args.y_frames, '2019-01-01', '2019-12-31')
partition = {'train': trainset, 'val':valset, 'test':testset}


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(partition, deepcopy(args))
        save_exp_result(setting, result)

In [ ]:
var1 = 'hid_dim'
var2 = 'n_layers'
df = load_exp_result('exp1')

plot_acc(var1, var2, df)
plot_loss_variation(var1, var2, df, sharey=False) #sharey를 True로 하면 모둔 subplot의 y축의 스케일이 같아집니다.
plot_acc_variation(var1, var2, df, margin_titles=True, sharey=True)

In [ ]:
df

In [ ]:
pwd

In [ ]:
# %mkdir results

In [ ]:
ls